# Assignment 1

- Ilse van Deventer 9996974
- Majdouline Hamdi 9767738
- Zexuan Li 8069182
- Zoé Ricardie 9107096
- Menno Zoetbrood 1084720

## Questions

a. Write a Python code that allows you to connect to the database file, send SQL queries to the database and extract the results.

b. Read the content of the customer relation (table) into Pandas DataFrame.

c. Using a similarity function that compares two records (similar to the one in the tutorial), report the customers with similarity > 0.7

In [ ]:
#Loading the necessary packages

import pandas as pd
import sqlite3
from sqlite3 import Error

In [ ]:
#creating a global connection to the database
conn = sqlite3.connect('assignment_1.db')
#creating a cursor
cur = conn.cursor()

In [ ]:
#Inspecting all tables from the database
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [row[0] for row in cur.fetchall()]

#Checking every tables' column types
for table in tables:
    print(f"\n=== Table: {table} ===")
    cur.execute(f"PRAGMA table_info({table})")
    for row in cur.fetchall():
      #making sure the text output is neatly aligned
      print(f"{'Column name: ' + row[1]:<40} {'Type: ' + row[2]:<15}")


=== Table: bill ===
Column name: bill_id                     Type: varchar(20)
Column name: branch_id                   Type: varchar(50)
Column name: amount                      Type: numeric(12,2)
Column name: customer_id                 Type: varchar(20)
Column name: bill_date                   Type: date     
Column name: due_date                    Type: date     
Column name: status                      Type: varchar(20)
Column name: payment_methods             Type: varchar(20)

=== Table: partner ===
Column name: partner_id                  Type: varchar(20)
Column name: company_name                Type: varchar(100)
Column name: country                     Type: varchar(20)
Column name: products_and_services       Type: varchar(50)
Column name: email                       Type: varchar(50)
Column name: industry                    Type: varchar(50)
Column name: is_active                   Type: boolean  

=== Table: product ===
Column name: product_id                  Type: va

In [ ]:
# A)

#Creating a function to connect to the database

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(f"Connection failed {e}")

    return conn

In [ ]:
#Sending the query to the database and receiving the resulting relation

def run_query(conn, query):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    # Send the query to the database
    cur.execute(query)
    # Extract the results of the query
    results = cur.fetchall()
    # Return the results
    return results

In [ ]:
#Creating a dataframe from the resulting relation

def convert_db_table_to_DF(conn, table):
    # get the names of the attributes in the database table
    header_query = "SELECT name FROM pragma_table_info('" + table + "') ORDER BY cid;"
    # print (header_query)
    cols_init = run_query(conn, header_query)
    cols = [cols_init[i][0] for i in range(len(cols_init))]
    # print(cols)
    # get the records of the table
    content_query = "Select * from " + table
    data = run_query(conn, content_query)
    df = pd.DataFrame(data, columns = cols)
    return df

In [ ]:
#Querying the database and getting the names of the tables in the database

database = "assignment_1.db"

#create a database connection
conn = create_connection(database)
with conn:
  query = 'Select name FROM sqlite_schema WHERE type = "table"'
  trading_data = run_query(conn, query)
  #checking if the database was properly connected
  print(trading_data)

[('bill',), ('partner',), ('product',), ('branch',), ('employee',), ('bill_product',), ('partner_product',), ('customer',)]


In [ ]:
# B)

database = "assignment_1.db"

#create a database connection
conn_trade = create_connection(database)
with conn_trade:
  tab_name = 'customer'
  customer = convert_db_table_to_DF(conn_trade, tab_name)

#display the content of the DataFrame
print(f"{customer} \n {'-' * 71}")

#checking whether the column type is the same as in the database
customer.info()

#there is no varchar type in python, so the column types will be stored as string,
#which Pandas.DataFrame.dtypes returns as 'object'

#changing the "create_date" column to datetime
customer["create_date"] = pd.to_datetime(customer["create_date"])
customer["date_of_birth"] = pd.to_datetime(customer["date_of_birth"])
#checking if the column type was correctly changed
print(f"{'-' * 71}")
customer.info()

  customer_id   name  gender     street house_number       city country  \
0        C001  Alice  Female   Street 1           10  Amsterdam      NL   
1        C002    Bob    Male   Street 2           20     Berlin      DE   
2        C003  Carol  Female   Street 3           30   New York      US   
3        C004   Dave    Male   Street 4           40     London      UK   
4        C005    Eve  Female   Street 5           50      Paris      FR   
5        C006  Alice  Female   Street 1           10  Amsterdam      NL   
6        C007  Grace  Female   Street 7           70     Berlin      DE   
7        C008  Heidi  Female   Street 8           80   New York      US   
8        C009   Ivan    Male   Street 9           90     London      UK   
9        C010   Judy  Female  Street 10          100      Paris      FR   

    phone_number               email customer_category date_of_birth  \
0           None    alice@example.nl           regular    1982-10-11   
1           None      bob@exam

In [ ]:
# C)

#create a function to compare how similar two sets are
def jaccard_similarity(set1, set2):
    intersection_size = len(set1.intersection(set2))
    union_size = len(set1.union(set2))
    return intersection_size / union_size if union_size != 0 else 0.0

#transform every row into a list of sets (one for each row)
row_sets = [set(customer.iloc[i].to_numpy()) for i in range(len(customer))]

#storing the results in a list
results = []

for i in range(len(row_sets)):
    for j in range(i + 1, len(row_sets)):
        sim = jaccard_similarity(row_sets[i], row_sets[j])
        results.append((i, j, sim))
        if sim > 0.7:
            print(f"Similarity between row {i+1} and row {j+1} = {sim:.3f} which is greater than 0.7")

#Row 0 and row 5 have a Jaccard similarity score above 0.7

Similarity between row 1 and row 6 = 0.714 which is greater than 0.7
